In [1]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2aFTy0XFQA6xSSUydVFoNC6LDF8_6XNAzJnWrMf2PPVBjjm47

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=6c4302f60f81363075c0a42db127e7aa265fe3f2841482a8de66040bcd25cbba
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00


In [3]:
from langchain.chains import LLMChain,SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate

from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap
import sys

In [4]:
model = AutoModelForCausalLM.from_pretrained("Trelis/Llama-2-7b-chat-hf-function-calling-v2",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Trelis/Llama-2-7b-chat-hf-function-calling-v2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/901 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [7]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """You are a vigilant digital forensics detective entrusted with the task of categorizing frauds based on detailed descriptions and analyzing spam emails or SMS to uncover the modus operandi of the scams. In addition to this, your role extends to identifying and plugging potential loopholes in the security measures that may have facilitated the fraudulent activity.

Upon receiving information about a fraud, categorize the type of fraud meticulously, considering various possibilities such as phishing, identity theft, financial scams, or any other fraudulent activities. Simultaneously, delve into the provided descriptions and associated spam emails or SMS to extract insights into the specific techniques employed by the fraudsters.

Go beyond identification and provide a thorough analysis of the scam's modus operandi, shedding light on the entry points, strategies, and recurring patterns. Your goal is to offer a comprehensive understanding of the fraud to aid in preventive measures and law enforcement efforts.

Furthermore, actively seek out and highlight potential loopholes in the existing security measures that may have been exploited by the fraudsters. Propose actionable recommendations to enhance security protocols and mitigate the risk of similar incidents occurring in the future.

Your responses should be detailed, clear, and focused on empowering individuals and organizations with the knowledge needed to not only understand and counteract the fraud but also to fortify their systems against future threats. Stay updated on emerging fraud tactics and evolving security measures to continually refine your analyses and loophole-plugging recommendations.

Your response should be structured and to the point such that it can be stored in a database, keeping that it mind, it should not contain any greetings because the output wont be used by a human.

"""

In [8]:
def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')


In [9]:
system_prompt = f"""
You are a vigilant digital forensics detective entrusted with the task of categorizing frauds based on detailed descriptions and analyzing spam emails or SMS to uncover the modus operandi of the scams. In addition to this, your role extends to identifying and plugging potential loopholes in the security measures that may have facilitated the fraudulent activity.

Upon receiving information about a fraud, categorize the type of fraud meticulously, considering various possibilities such as phishing, identity theft, financial scams, or any other fraudulent activities. Simultaneously, delve into the provided descriptions and associated spam emails or SMS to extract insights into the specific techniques employed by the fraudsters.

Go beyond identification and provide a thorough analysis of the scam's modus operandi, shedding light on the entry points, strategies, and recurring patterns. Your goal is to offer a comprehensive understanding of the fraud to aid in preventive measures and law enforcement efforts.

Furthermore, actively seek out and highlight potential loopholes in the existing security measures that may have been exploited by the fraudsters. Propose actionable recommendations to enhance security protocols and mitigate the risk of similar incidents occurring in the future.

Your responses should be detailed, clear, and focused on empowering individuals and organizations with the knowledge needed to not only understand and counteract the fraud but also to fortify their systems against future threats. Stay updated on emerging fraud tactics and evolving security measures to continually refine your analyses and loophole-plugging recommendations.

Your response should be structured and to the point such that it can be stored in a database, keeping that it mind, it should not contain any greetings because the output wont be used by a human.

Put your answer in the json format with category and and analysis key value pair given below.

```json

"category": "category of the fraud",
"analysis": "analysis of the fraud"


```

The output should only contain json and nothing else. You will be penalised if there is no json.
Make sure the json is parsable
"""

instruction = "The output should only contain json and nothing else. You will be penalised if there is no json. Answer in json based on the following fraud details  \n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>

You are a vigilant digital forensics detective entrusted with the task of categorizing frauds based on detailed descriptions and analyzing spam emails or SMS to uncover the modus operandi of the scams. In addition to this, your role extends to identifying and plugging potential loopholes in the security measures that may have facilitated the fraudulent activity.

Upon receiving information about a fraud, categorize the type of fraud meticulously, considering various possibilities such as phishing, identity theft, financial scams, or any other fraudulent activities. Simultaneously, delve into the provided descriptions and associated spam emails or SMS to extract insights into the specific techniques employed by the fraudsters.

Go beyond identification and provide a thorough analysis of the scam's modus operandi, shedding light on the entry points, strategies, and recurring patterns. Your goal is to offer a comprehensive understanding of the fraud to aid in preventive measures

In [10]:
prompt = PromptTemplate(template=template, input_variables=["text"])
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 3000, 'top_k' :200})
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)

In [11]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 3000, 'top_k' :200})

In [12]:
description = """Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed å£1000 cash or å£5000 prize!"""

In [13]:
response = llm_chain.run(description)
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


  {
"category": "Phishing",
"analysis": "The fraudster is using a fake prize draw to trick the victim into calling a number to claim their prize. The number may be a toll or premium rate number, resulting in the victim incurring charges. The fraudster may also request personal information or payment to claim the prize, which could lead to identity theft or financial loss."
}


In [14]:
import re
import json

def parse_json_string(incomplete_json_str):
    pattern = r'"category"\s*:\s*"([^"]*)"\s*,\s*"analysis"\s*:\s*"([^"]*)"'
    match = re.search(pattern, incomplete_json_str)

    if match:
        category = match.group(1)
        analysis = match.group(2)
        result_json = {
            "category": category,
            "analysis": analysis
        }

        return result_json
    else:
        return None

parsed_result = parse_json_string(response)

if parsed_result:
    print(json.dumps(parsed_result, indent=2))
else:
    print("Failed to parse the JSON string.")



{
  "category": "Phishing",
  "analysis": "The fraudster is using a fake prize draw to trick the victim into calling a number to claim their prize. The number may be a toll or premium rate number, resulting in the victim incurring charges. The fraudster may also request personal information or payment to claim the prize, which could lead to identity theft or financial loss."
}


In [15]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [16]:
!pip install -U flask-cors

In [17]:
# from pydantic import BaseModel, ValidationError
# import json
# class Analysis(BaseModel):
#     category: str
#     analysis: str

In [18]:
# max_tries = 0

# while(max_tries < 3):
#   try:
#     json_objects = json.loads(completion)
#     validated_data = Analysis.parse_obj(json_objects)
#     break
#   except:
#     completion = handle_validation_failure("Output Not json", prompt, max_tries)
#     max_tries+=1


In [19]:
# validated_data

In [20]:
# failed = []
# validated = []


# validated_data = Analysis.parse_obj(json_objects)


# print(failed)

In [21]:
# print(validated_data)

In [ ]:
from flask import Flask,jsonify, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
import logging
app = Flask(__name__)
logging.basicConfig(level=logging.INFO)
logging.getLogger('flask_cors').level = logging.DEBUG
CORS(app)
run_with_ngrok(app)


@app.route("/analysis", methods=["POST"])
def home():
    data = request.get_json()
    message = data.get("message")

    if message is None:
        return jsonify({"error": "message parameter is missing"}), 400

    response = llm_chain.run(message)
    parsed_result = parse_json_string(response)
    print(parsed_result)

    try:
      response = json.dumps(parsed_result)
      return jsonify(parsed_result)
    except:
      return jsonify({"Error":"json could not be created"})

app.run()

DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://09d7-35-197-124-135.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


DEBUG:flask_cors.extension:Request to '/' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
DEBUG:flask_cors.extension:CORS have been already evaluated, skipping
INFO:werkzeug:127.0.0.1 - - [17/Jan/2024 11:28:18] "GET / HTTP/1.1" 404 -
DEBUG:flask_cors.extension:Request to '/analysis' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*'

{'category': 'Phishing', 'analysis': 'The fraudsters are using a classic phishing tactic to lure victims into divulging sensitive information by offering free bonuses on a Rummy Circle platform. The email or SMS may have included a sense of urgency or scarcity to prompt immediate action. The entry point may have been a fake promotion or a misleading message. The modus operandi involves creating a sense of trust or urgency to elicit personal information or financial details. Potential loopholes in security measures may include lax verification processes or a lack of awareness regarding phishing tactics.'}


DEBUG:flask_cors.extension:Request to '/analysis' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
INFO:werkzeug:127.0.0.1 - - [17/Jan/2024 11:47:06] "POST /analysis HTTP/1.1" 200 -


{'category': 'financial scams', 'analysis': 'The scam utilized a phishing tactic to entice the victim into clicking on a malicious link, which led to the theft of their personal information. The scammers may have used the stolen data to commit identity theft or financial fraud. To prevent such incidents, individuals should be cautious when clicking on links or providing personal information online, and use strong and unique passwords for their accounts.'}
